In [2]:
! pip install pandas transformers
import json
import pandas as pd
from transformers import AutoTokenizer

with open('../labeled_data/gold_labels/reconstructed_gold_labels.json', 'r') as f:
    data = json.load(f)

texts = [item['text'] for item in data]
entities = [item['entities'] for item in data]
df = pd.DataFrame({'text': texts, 'entities': entities})

   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.6 MB 6.3 MB/s eta 0:00:02
   --------- ------------------------------ 2.9/11.6 MB 8.8 MB/s eta 0:00:01
   -------------------- ------------------- 6.0/11.6 MB 11.5 MB/s eta 0:00:01
   ---------------------------- ----------- 8.4/11.6 MB 11.3 MB/s eta 0:00:01
   ---------------------------------- ----- 10.0/11.6 MB 10.5 MB/s eta 0:00:01
   ------------------------------------- -- 10.7/11.6 MB 9.7 MB/s eta 0:00:01
   -------------------------------------- - 11.3/11.6 MB 8.8 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 7.8 MB/s eta 0:00:00


c:\Users\magnu\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [39]:
def find_nl_positions(text):
    nl_pos = []
    current = 0
    while text.find("\n", current) != -1:
        nl_pos.append(text.find("\n", current))
        current = text.find("\n", current) +1
    return nl_pos

In [40]:
df['nl_positions'] = df['text'].apply(
    lambda x: find_nl_positions(x)
)

In [41]:
df

,text,entities,tokenized,labels,nl_positions
0,A new ransomware-as-a-service (RaaS) operation...,"[{'start': 716, 'end': 732, 'type': 'ORG', 'te...","[input_ids, token_type_ids, attention_mask, of...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[162, 359, 532, 702, 869, 1140, 1238, 1434, 14..."
1,The popular Docker-OSX project has been remove...,"[{'start': 14, 'end': 24, 'type': 'Software', ...","[input_ids, token_type_ids, attention_mask, of...","[O, O, O, O, B-Software, I-Software, I-Softwar...","[182, 410, 580, 701, 847, 1101, 1287, 1444, 16..."
2,A former core infrastructure engineer at an in...,"[{'start': 82, 'end': 97, 'type': 'LOC', 'text...","[input_ids, token_type_ids, attention_mask, of...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-LOC,...","[220, 523, 751, 1135, 1349, 1721, 2063, 2263, ..."
3,Threat actors target Middle Eastern organizati...,"[{'start': 23, 'end': 37, 'type': 'LOC', 'text...","[input_ids, token_type_ids, attention_mask, of...","[O, O, O, O, O, O, O, B-LOC, O, O, O, O, O, O,...","[219, 519, 681, 870, 1012, 1163, 1368, 1493, 1..."
4,"Since surfacing in February 2024, RansomHub ra...","[{'start': 37, 'end': 46, 'type': 'MAL-ORG', '...","[input_ids, token_type_ids, attention_mask, of...","[O, O, O, O, O, O, O, O, O, O, O, B-MAL-ORG, I...","[155, 528, 914, 1040, 1298, 1546, 1982, 1983, ..."
...,...,...,...,...,...
56,​Russian law enforcement detained almost 100 s...,"[{'start': 2, 'end': 10, 'type': 'LOC', 'text'...","[input_ids, token_type_ids, attention_mask, of...","[O, O, O, B-Event, I-Event, I-Event, I-Event, ...","[238, 463, 786, 1075, 1240, 1471, 1718, 2024, ..."
57,The Russian state-sponsored APT29 hacking grou...,"[{'start': 5, 'end': 13, 'type': 'LOC', 'text'...","[input_ids, token_type_ids, attention_mask, of...","[O, O, O, O, O, O, O, B-MAL-ORG, I-MAL-ORG, O,...","[208, 388, 523, 734, 971, 1081, 1312, 1485, 16..."
58,The South Korea-aligned cyberespionage group A...,"[{'start': 6, 'end': 25, 'type': 'LOC', 'text'...","[input_ids, token_type_ids, attention_mask, of...","[O, O, O, B-LOC, I-LOC, I-LOC, I-LOC, O, O, O,...","[206, 365, 570, 798, 973, 1177, 1331, 1516, 17..."
59,The Federal Communications Commission (FCC) an...,"[{'start': 6, 'end': 39, 'type': 'ORG', 'text'...","[input_ids, token_type_ids, attention_mask, of...","[O, O, O, B-ORG, I-ORG, I-ORG, O, B-ORG, O, B-...","[195, 435, 609, 830, 1055, 1224, 1449, 1868, 2..."


In [42]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
df['tokenized'] = df['text'].apply(
    lambda x: tokenizer(x, return_offsets_mapping=True, truncation=True, padding=True)
)

In [43]:
def align_labels_to_tokens(text, entities, tokenized, nl_positions):
    offset_mapping = tokenized['offset_mapping']
    labels = ["O"] * len(offset_mapping) # Initialize all tokens with "O"

    for entity in entities:
        start, end, label_type = entity['start'], entity['end'], entity['type']
        nls_before_entity = 0
        for nl_pos in nl_positions:
            if nl_pos < start:
                nls_before_entity +=1
            else:
                break
        start -= nls_before_entity
        end -= nls_before_entity
        entity_started = False
        #print(entity)

        for idx, (token_start, token_end) in enumerate(offset_mapping):
            #print(token_start, token_end)
            if token_start is None or token_end is None:
                continue
            if token_start >= start and token_end <= end:
                if entity_started:
                    labels[idx] = f"I-{label_type}"
                else:
                    labels[idx] = f"B-{label_type}"
                    entity_started = True
            else:
                entity_started = False

    return labels

df['labels'] = df.apply(lambda row: align_labels_to_tokens(row['text'], row['entities'], row['tokenized'], row['nl_positions']), axis=1)

In [44]:
# Save to JSON or CSV format
output_data = []
for _, row in df.iterrows():
    tokens = tokenizer.convert_ids_to_tokens(row['tokenized']['input_ids'])
    labels = row['labels']
    output_data.append({'tokens': tokens, 'labels': labels})

# Save the processed data
with open('tokenized_ner_data_3.json', 'w') as f:
    json.dump(output_data, f, indent=4)

: 

In [8]:
for i, label in enumerate(output_data[0]['labels']):
    if label != "O":
        print(output_data[0]['tokens'][i])

C
##ica
##da
##33
##01
##da
##33
##01
Ra
##ping
##C
##om
##pute
##r
is
##da
##33
##01
double
-
ex
##tor
##tion
tactics
where
they
breach
corporate
networks
,
steal
data
,
and
then
en
##c
##ry
##pt
devices
.
The
##01
and
AL
##C
,
core
from
one
also
ransom
##net
for
##inet
Pa
##lo
and
down
a
R
##ust
-


In [ ]:
def find_nl_positions(text):
    nl_pos = []
    current = 0
    while text.find("\n", current) != -1:
        nl_pos.append(text.find("\n", current))
        current = text.find("\n", current) +1
    return nl_pos

In [30]:
nl_pos

[162,
 359,
 532,
 702,
 869,
 1140,
 1238,
 1434,
 1467,
 1683,
 1996,
 2172,
 2388,
 2671,
 2850,
 3076,
 3294,
 3624,
 3755,
 3886,
 4049,
 4273,
 4457,
 4690,
 4747,
 4806,
 4868,
 4939]

In [37]:
labels = align_labels_to_tokens(text, df.iloc[0]['entities'], df.iloc[0]['tokenized'], nl_pos)

In [38]:
for i, label in enumerate(labels):
    if label != "O":
        print(output_data[0]['tokens'][i])

C
##ica
##da
##33
##01
C
##ica
##da
##33
##01
B
##lee
##ping
##C
##om
##pute
##r
C
##ica
##da
C
##ica
##da
##33
##01
conducts
double
-
ex
##tor
##tion
tactics
where
they
breach
corporate
networks
,
steal
data
,
and
then
en
##c
##ry
##pt
devices
True
##se
##c
C
##ica
##da
##33
##01
AL
##P
##H
##V
Black
##C
##at
AL
##P
##H
##V
'
s
AL
##P
##H
##V
FBI
Change
Healthcare
True
##se
##c
C
##ica
##da
##33
##01
B
##ru
##tus
b
##ot
##net
C
##isco
Fort
##inet
Pa
##lo
Alto
Sonic
##W
##all
B
##ru
##tus
AL
##P
##H
##V
C
##ica
##da
##33
##01
